# "Generic Machine Learning" 
> "Code for generic ML" 

- toc:false
- branch: master
- badges: true
- comments: true
- author: Mun Fai Chan
- categories: [fastpages, jupyter]



This notebook provides code to Generic Machine Learning Inference on Heterogenous Treatment Effects in Randomized Experiments by Victor Chernozhukov, Mert Demirer, Esther Duflo, and Iván Fernández-Val. 

https://arxiv.org/abs/1712.04802

### References 
https://github.com/arnaudfrn/MLheterogeneity/blob/dev/src/vb_heterogeneity_FE.R

Many thanks to Arnaud Fournier, who provided the R code for this.  

Author of notebook : Mun Fai Chan

####  Future developments for code

1. Hyperparemeter tuning on ML estimators
2. Converting pandas dataframes to LaTex tables. 
3. Aesthetic updates - includes adding astericks for significance 
4. Add in fixed effects 

#### Other developments
1. Empirical application
2. Monte Carlo simulation to test veracity and robustness of code

In [1]:
from propscore import PropensityScore
import random
import pandas as pd
import sklearn
import sklearn.model_selection
import numpy as np
import statistics as stats
import statsmodels.api as sm
from scipy.stats import norm

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
from causalinference import CausalModel
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn import datasets, ensemble
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet

## Data

In [8]:
df = pd.read_csv("~/OneDrive - London School of Economics/LSE/Year 3/EC331/November/simdata1.csv")
# In this simulated dataset, all controls are uniformly random around (-1,1). Treatment (binary) is randomly assigned
# and has a treatment effect of 2.0 + some gaussian noise. 

controls = ['X1','X2','X3','X4','X5']
treatment = 'treatment'

In [9]:
ps = PropensityScore(treatment, controls, df);
df = df.join(ps.propscore)

df.head()

                           Logit Regression Results                           
Dep. Variable:              treatment   No. Observations:                 2000
Model:                          Logit   Df Residuals:                     1996
Method:                           MLE   Df Model:                            3
Date:                Mon, 04 Jan 2021   Pseudo R-squ.:                0.003082
Time:                        14:12:22   Log-Likelihood:                -1382.0
converged:                       True   LL-Null:                       -1386.3
Covariance Type:            nonrobust   LLR p-value:                   0.03598
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
X4             0.0656      0.044      1.481      0.138      -0.021       0.152
X1XX2         -0.0862      0.046     -1.889      0.059      -0.176       0.003
X4_sq          0.0576      0.032      1.802      0.0

,X1,X2,X3,X4,X5,treatment,outcome,propscore
0,0.222325,1.096420,0.791501,-0.047723,-0.638967,1,1.896553,0.479204
1,-0.333924,-2.472571,-0.635518,0.640664,0.408980,0,-0.005217,0.483821
2,-0.635914,-0.714505,0.170289,-0.591279,-0.211143,0,0.097441,0.470775
3,0.618626,-1.055120,0.486168,-0.441626,-2.865479,0,0.015330,0.494820
4,0.807180,1.475266,-0.033577,0.354557,0.868843,1,1.893319,0.467209


###  Initialisation 

In [115]:
iterations = 100
k = 5 # number of groups for heterogeneity analysis
alpha = 0.05 # significance level 

## Running everything

In [126]:
ML_models = ["random_forest", "SVM", "gradient_boost", "neural_net", "ElasticNet"]

for x in ML_models: 
    summary = Generic_ML_single(df, controls, 10, x, alpha , 5) 
    print (str(x) + ": lamda1: " + str(summary[-2])+ " lambda2: " + str(summary[-1]))

random_forest: lamda1: -5.765410377113008e-05 lambda2: 0.7962556067959392
SVM: lamda1: 6.37687223615527e-05 lambda2: 0.7990199258698686
gradient_boost: lamda1: -5.810927679997994e-06 lambda2: 0.7972126884049128
neural_net: lamda1: -0.00013872698846416478 lambda2: 0.7969536495255202
ElasticNet: lamda1: 0.00038697229802813827 lambda2: 0.7977092031809006


This allows us to quickly compare between different ML estimators. In particular, we want to minimise lambda1 and lambda2. 

In [116]:
summary = Generic_ML_single(df, controls, iterations, "random_forest", alpha , 5) 

In [128]:
BLP = summary[0]; BLP

,ATE,HET
coeff,2.002647,0.019719
se,0.007427,0.074006
pvalue,0.000000,0.987714
lower bound,1.987861,-0.120732
upper bound,2.017433,0.162308


In [130]:
GATES = summary[1]; GATES

,G1,G5,G1 - G5
coeff,1.995021,2.010402,-0.012300
se,0.017296,0.017322,0.024500
pvalue,0.000000,0.000000,1.000000
lower bound,1.960978,1.976616,-0.061299
upper bound,2.029177,2.044189,0.036699


In [131]:
CLAN = summary[2]; CLAN

,coeff,se,pvalue,lower bound,upper bound
Most affected X1,-0.046367,0.079034,0.393447,-0.202160,0.109426
Least affected X1,-0.025723,0.079034,0.456220,-0.181875,0.130428
Most - least affected X1,0.033200,0.250000,0.693000,-0.460711,0.527111
Most affected X2,0.026326,0.083156,0.602332,-0.136445,0.189097
Least affected X2,-0.007814,0.083156,0.345540,-0.170585,0.154957
Most - least affected X2,0.034150,0.131500,0.620500,-0.222605,0.290905
Most affected X3,0.051797,0.081971,0.473087,-0.112999,0.216593
Least affected X3,-0.010720,0.081971,0.637616,-0.172631,0.151190
Most - least affected X3,0.049050,0.130000,0.542000,-0.207705,0.305805
Most affected X4,0.015797,0.083432,0.578876,-0.147815,0.179409


# HELPER FUNCTIONS

#### BLP 

In [12]:
def BLP(df, alpha): 
    '''
    Returns summary results, whose parameters can be used to obtain BLP of CATE. 
    Contains: 
        Estimator Coefficients of Term 2 and 3 
        Standard Error 
        p values 
        Confidence Interval (lower and upper bounds)
    
    Returns lambda1 - value to help choose the best ML method 
        
    '''
    
    term2 = df['treatment'] - df['propscore']
    S = df['S']
    term3 = term2 * (S - np.mean(S))
    
    
    combined = df.copy()
    combined.loc[:,'term2'] = term2 
    combined.loc[:,'term3'] = term3
    combined.loc[:,'ones'] = 1 
    
    X_reg = combined[['B', 'S', 'ones', 'term2', 'term3']]
    y = combined[['outcome']]
    
    regBLP = sm.OLS(y, X_reg)
    res_BLP = regBLP.fit()
    
    res_BLP = results_summary_to_dataframe(res_BLP, alpha)
    
    lambda1 = res_BLP.iloc[-1,0] * stats.variance(S)   
    return res_BLP, lambda1
    
def results_summary_to_dataframe(results, alpha):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    pvals = results.pvalues
    coeff = results.params
    std_err = results.bse
    
    crit_val = norm.ppf(1-alpha/2) 

    lb = coeff - std_err * crit_val
    ub = coeff + std_err * crit_val
    

    results_df = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff,
                               "lb":lb,
                               "ub":ub,
                               "std_err":std_err, 
                                })

    #Reordering...
    results_df = results_df[["coeff","std_err","pvals","lb","ub"]]
    return results_df

def BLP_to_storage(res_BLP):
    
    '''
    Takes the output of BLP and store them as lists, whereby the output refers to: 
        res_BLP - summary table containing parameters to construct BLP, along with their p-values, standard errors and lower and upper bounds
        
    Returns 2 lists data_HET and data_ATE whose array-equivalent is of dimension (1 variable, 5 attributes)
    '''
    
    # HET parameter 
    HET = res_BLP.iloc[-1,0]
    HET_se = res_BLP.iloc[-1,1]
    HET_pvals = res_BLP.iloc[-1, 2]
    HET_lb = res_BLP.iloc[-1, 3]
    HET_ub = res_BLP.iloc[-1, 4]

    # ATE 
    ATE = res_BLP.iloc[-2,0]
    ATE_se = res_BLP.iloc[-2,1]
    ATE_pvals = res_BLP.iloc[-2,2]
    ATE_lb = res_BLP.iloc[-2,3]
    ATE_ub = res_BLP.iloc[-2,4]
    
    # Storage
    
    data_HET = [HET, HET_se, HET_pvals, HET_lb, HET_ub]
    data_ATE = [ATE, ATE_se, ATE_pvals, ATE_lb, ATE_ub]

    return data_HET, data_ATE

#### GATES

In [111]:
def GATES(df, k , alpha): 
    '''
    Returns summary statistics, whose results can give us the average treatment effect 
    for most and least affected group. 
    
    Contains: 
        Estimator Coefficients  
        Standard Error 
        p values 
        Confidence Interval (lower and upper bounds)
        
    Returns lambda2 - value to help choose the best ML method 
    
    Parameters 
    ----------
    df -- (main) dataframe which must contain the following items: 
        propensity score 
        B - proxy predictor for BCA 
        S - proxy predictor for CATE
        treatment 
        
    k -- number of groups 
    '''
    
    combined = df.copy()
    term2 = df['treatment'] - df['propscore']
    combined.loc[:,'term2'] = term2
    combined.loc[:,'ones'] = 1
    
    groups = groups_multiply(df, group_create(k, df), k)
    combined = pd.concat([combined,groups], axis = 1) 
  
    controls = ["B", "S", "ones"] + ["G" + str(i) for i in range(1,k+1)]
    X_GATES = combined[controls] # modify for auto selection of columns
    y = combined[['outcome']]
    
    regGATES = sm.OLS(y, X_GATES)
    res_GATES = regGATES.fit()
    
    # Hypothesis testing 
    hypothesis = "(G1 = " + "G" + str(k) + ")" # G1 = G{k}
    t_test_html = res_GATES.t_test(hypothesis).summary().as_html()
    t_test = pd.read_html(t_test_html, header=0, index_col=0)[0]
    
    res_GATES = results_summary_to_dataframe(res_GATES, alpha)
    
    lambda2 = res_GATES.iloc[3:, 0].mean()**2 / k
    
    return res_GATES, t_test, lambda2
    

def group_create(k, df): 
    '''
    Returns quantiles of the variable 'S', encoded into dummy variables
    '''
    breaks = df['S'].quantile(np.linspace(0,1,(k+1)))
    breaks.iloc[0,] = breaks.iloc[0,] - 0.001 
    breaks.iloc[k,] = breaks.iloc[k,] - 0.001 
    
    combined = df.copy()
    combined['Groups'] = pd.cut(x= df['S'], bins = breaks) # this will fail if there are too many groups
    groups = pd.get_dummies(combined['Groups'])
    
    return groups

def groups_multiply(df, groups, k):
    '''
    Multiply groups dataframe with term 2 and rename columns 
    '''
    
    combined = df.copy()
    term2 = df['treatment'] - df['propscore']
    combined.loc[:,'term2'] = term2
    
    groups = np.multiply(groups, combined['term2'].values.reshape(len(df.index),1))
    groups.columns = ["G" + str(i) for i in range(1,k+1)] 
    
    return groups

def GATES_to_storage(res_GATES, t_test_GATES, alpha):
    
    '''
    Takes the output of GATES and store them as lists, whereby the output refers to: 
        res_GATES - summary table containing parameters to construct GATES, along with their p-values and standard errors 
        t_test_GATEs - t test table to determine if G1 = Gk 
    
    Returns a list whose array-equivalent is dimension of (# of variables, # of attributes )
    '''
    
    # Most affected group 
    gamma1 = res_GATES.iloc[3,0]
    gamma1_se = res_GATES.iloc[3,1]
    gamma1_pvals = res_GATES.iloc[3,2]
    gamma1_lb = res_GATES.iloc[3,3]
    gamma1_ub = res_GATES.iloc[3,4]

    # Least affected group 
    gammak = res_GATES.iloc[-1,0]
    gammak_se = res_GATES.iloc[-1,1]
    gammak_pvals = res_GATES.iloc[-1,2]
    gammak_lb = res_GATES.iloc[-1,3]
    gammak_ub = res_GATES.iloc[-1,4]
    
    # Difference between most and least affected group 
  
    crit_val = norm.ppf(1-alpha/2) 

    gamma_diff = t_test_GATES.iloc[0,0]
    gamma_diff_se = t_test_GATES.iloc[0,1]
    gamma_diff_pvals = t_test_GATES.iloc[0,3] 
    gamma_diff_lb = gamma_diff - crit_val * gamma_diff_se
    gamma_diff_ub = gamma_diff + crit_val * gamma_diff_se
    
    data_gamma1 = [gamma1, gamma1_se, gamma1_pvals, gamma1_lb, gamma1_ub]
    data_gammak = [gammak, gammak_se, gammak_pvals, gammak_lb, gammak_ub]
    data_gamma_diff = [gamma_diff, gamma_diff_se, gamma_diff_pvals, gamma_diff_lb, gamma_diff_ub]

    data_gamma = [data_gamma1, data_gammak, data_gamma_diff]
    
    return data_gamma

#### CLAN

In [25]:
def CLAN(df, controls, k = 5, alpha):
    data_CLAN_loop = []
    for x in controls: 
        res_CLAN, t_test = CLAN_single(df, x, k)
        data_CLAN = CLAN_to_storage(res_CLAN, t_test, alpha)
        data_CLAN_loop.append(data_CLAN)
    return data_CLAN_loop

def CLAN_single(df, control, k = 5):
    '''
    Returns the average characteristic for one control between the most and least affected groups 
    
    '''
    threshold = 1/k
    high_effect = df['S'].quantile(1 - threshold)
    low_effect = df['S'].quantile(threshold)
    
    combined = df.copy()
    combined.loc[:,'high'] = (combined.loc[:,"S"] > high_effect).astype(int) # dummy variables for high 
    combined.loc[:,'low'] = (combined.loc[:,"S"] > low_effect).astype(int) # dummy variables for low 
    combined.loc[:,'minusones'] = -1
    
    X_control = combined[['high', 'low', 'minusones']] # I have no idea why I included minusones 
    y_control = combined[[control]]
    
    reg_CLAN = sm.OLS(y_control, X_control)
    res_CLAN = reg_CLAN.fit()

    hypothesis = "(high = low)" 
    t_test_html = res_CLAN.t_test(hypothesis).summary().as_html()
    t_test = pd.read_html(t_test_html, header=0, index_col=0)[0]

    res_CLAN = results_summary_to_dataframe(res_CLAN, alpha)
    
    return res_CLAN, t_test

def CLAN_to_storage(res_CLAN, t_test, alpha):
    '''
    Takes the summary results of CLAN and its t test and store them as lists 
    '''
    h_coeff = res_CLAN.iloc[0,0]
    h_se = res_CLAN.iloc[0,1]
    h_pvals = res_CLAN.iloc[0,2]
    h_lb = res_CLAN.iloc[0,3]
    h_ub = res_CLAN.iloc[0,4]
    data_h = [h_coeff, h_se, h_pvals, h_lb, h_ub]
    
    l_coeff = res_CLAN.iloc[1,0]
    l_se = res_CLAN.iloc[1,1]
    l_pvals = res_CLAN.iloc[1,2]
    l_lb = res_CLAN.iloc[1,3]
    l_ub = res_CLAN.iloc[1,4]
    data_l = [l_coeff, l_se, l_pvals, l_lb, l_ub]
    
    crit_val = norm.ppf(1-alpha/2) 
    
    diff_coeff = t_test.iloc[0,0]
    diff_se = t_test.iloc[0,1]
    diff_pvals = t_test.iloc[0,3]
    diff_lb = diff_coeff - crit_val * diff_se
    diff_ub = diff_coeff + crit_val * diff_se
    data_diff = [diff_coeff, diff_se, diff_pvals, diff_lb, diff_ub]
    
    data_CLAN = data_h, data_l, data_diff
    return data_CLAN

#### Converting data into dataframes

In [27]:
def data_BLP_to_df(data_HET_loop, data_ATE_loop): 
    '''
    Takes the data of BLP stored as a list, find its median over different iterations and adjusts p values. 
    
    Returns it as a dataframe 
    '''
    
    data_HET_array = np.array(data_HET_loop)
    data_HET_final = np.median(data_HET_array, axis = 0)
    data_HET_final[2] = np.minimum(1, data_HET_final[2] *2)

    data_ATE_array = np.array(data_ATE_loop)
    data_ATE_final = np.median(data_ATE_array, axis = 0)
    data_ATE_final[2] = np.minimum(1, data_ATE_final[2] * 2)   
    
    df_ATE = pd.DataFrame(data_ATE_final, 
                     index = ['coeff', 'se', 'pvalue', 'lower bound', 'upper bound'], 
                     columns = ['ATE'])

    df_HET = pd.DataFrame(data_HET_final, 
                     index = ['coeff', 'se', 'pvalue', 'lower bound', 'upper bound'], 
                     columns = ['HET'])

    frames = [df_ATE, df_HET]
    
    df_BLP = pd.concat(frames, axis = 1)
    
    return df_BLP
    
def data_GATES_to_df(data_GATES_loop, groups): 
    '''
    Takes the data of GATES stored as a list, find its median over different iterations and adjusts p values. 
    
    Returns it as a dataframe 
    '''
    
    # GATES 
    data_GATES_array = np.array(data_GATES_loop)
    data_GATES_final = np.median(data_GATES_array, axis = 0)
    data_GATES_final[:, 2] = np.minimum(1, data_GATES_final[:, 2]* 2)
    
    df_GATES = pd.DataFrame(data_GATES_final, 
                        columns = ['coeff', 'se', 'pvalue', 'lower bound', 'upper bound'], 
                        index = ['G1', "G" + str(groups), "G1 - G" + str(groups)])
    
    return df_GATES.transpose()

def data_CLAN_to_df(data_CLAN_loop, controls = controls): 
    '''
    Takes the data of GATES stored as a list, find its median over different iterations and adjusts p values. 
    
    Returns it as a dataframe 
    '''
    
    # CLAN 
    data_CLAN_array = np.array(data_CLAN_loop) 


    data_CLAN_final = np.median(data_CLAN_array, axis = 0) # This code is technically wrong as we take the upper medians for the lower bounds
    data_CLAN_final[0,2,:] = np.minimum(1, data_CLAN_final[0,2,:] * 2)
    
    list = []
    for x in controls: 
        list1 = ['Most affected ' + str(x), 'Least affected ' + str(x), 'Most - least affected ' + str(x) ]
        list.append(list1)
    
    flattened_list = [y for x in list for y in x]

    data_CLAN_new = data_CLAN_final.reshape(-1,5)
    df_CLAN = pd.DataFrame(data_CLAN_new, 
                      columns = ['coeff', 'se', 'pvalue', 'lower bound', 'upper bound'], 
                      index = flattened_list)

    return df_CLAN

#### Putting everything together

In [55]:
def Generic_ML_single(df, controls, iterations = 10, model = "random_forest", alpha = 0.05, k = 5): 
    '''
    Runs the whole generic ML algorithm for a ML model and returns a list of datasets for all parameters.  
    '''
    
    data_HET_loop = []
    data_ATE_loop = []
    lambda1_loop = []

    data_GATES_loop = []
    lambda2_loop = []

    data_CLAN_loop = []


    for x in range(iterations): 
        main, aux = sklearn.model_selection.train_test_split(df, train_size = 0.5, random_state = x)
        main2 = ML_estimator(main, aux, model) 
    
        # BLP
        res_BLP, lambda1 = BLP(main2)
        data_HET, data_ATE = BLP_to_storage(res_BLP)
        data_HET_loop.append(data_HET)
        data_ATE_loop.append(data_ATE)
        lambda1_loop.append(lambda1)
    
        #GATES
        res_GATES, t_test_GATES, lambda2 = GATES(main2, k, alpha) 
        data_GATES = GATES_to_storage(res_GATES, t_test_GATES, alpha)
        data_GATES_loop.append(data_GATES)
        lambda2_loop.append(lambda2)
    
        # CLAN
        controls = controls 
        data_CLAN = CLAN(main2, controls)
        data_CLAN_loop.append(data_CLAN)

        # BLP
        data_HET_array = np.array(data_HET_loop)
        data_HET_final = np.median(data_HET_array, axis = 0)
        data_HET_final[2] = np.minimum(1, data_HET_final[2] *2)

        data_ATE_array = np.array(data_ATE_loop)
        data_ATE_final = np.median(data_ATE_array, axis = 0)
        data_ATE_final[2] = np.minimum(1, data_ATE_final[2] * 2)    
    
    
    df_BLP = data_BLP_to_df(data_HET_loop, data_ATE_loop)
    df_GATES = data_GATES_to_df(data_GATES_loop, k)
    df_CLAN = data_CLAN_to_df(data_CLAN_loop, controls = controls)
    
    lambda1 = np.mean(lambda1_loop)
    lamda2 = np.mean(lambda2_loop)
    
    summary = [df_BLP, df_GATES, df_CLAN, lambda1, lambda2]
    return summary

#### ML estimators

In [19]:
def ML_estimator(main, aux, model):
    '''
    Returns the main dataset combined with B and S, which are proxy predictors for BCA and CATE respectively 
    
    Parameters 
    ----------
    main: main dataset which must contain treatment and outcome
    aux: auxilliary dataset which must contain treatment and outcome
    model - in string format 
        models = ["random_forest", "SVM", "gradient_boost", "neural_net", "ElasticNet"]
    
    # need to set the seed of the ML_estimators
    
    '''
    
    # Initialization
    aux0 = aux[aux['treatment'] == 0]
    aux1 = aux[aux['treatment'] == 1]
    X_aux0 = aux0[['treatment', 'X1', 'X2', 'X3', 'X4', 'X5']]
    y_aux0 =aux0['outcome']
    X_aux1 = aux1[['treatment', 'X1', 'X2', 'X3', 'X4', 'X5']]
    y_aux1 =aux1['outcome']
    
    X_main = main[['treatment', 'X1', 'X2', 'X3', 'X4', 'X5']]
    y_main = main['outcome']
    
    # Model 
    if model == "random_forest": 
        combined = random_forest(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1)
    elif model == "SVM": 
        combined = SVM(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1)
    elif model == "gradient_boost": 
        combined = gradient_boost(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1)
    elif model == "neural_net": 
        combined = neural_net(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1)
    elif model == "ElasticNet": 
        combined = ElasticNet(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1)
    
    # Add variance
    if stats.variance(combined['S']) == 0 : 
        combined['S'] = combined['S'] + np.random.normal(0,0.1, len(combined['S'])) 
    if stats.variance(combined['B']) == 0 : 
        combined['B'] = combined['B'] + np.random.normal(0,0.1, len(combined['B'])) 
        
    return combined

def random_forest(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1):
    
    # Model 
    clf = RandomForestRegressor(max_depth=2, random_state=0)
    
    clf.fit(X_aux0, y_aux0)
    B = clf.predict(X_main)

    clf.fit(X_aux1, y_aux1)
    clf.predict(X_main)
    S = clf.predict(X_main) - B 
    
    combined = main.copy()
    combined['B'] = B 
    combined['S'] = S
        
    return combined

def SVM(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1):

    # Model 
    clf = svm.SVR()
    
    clf.fit(X_aux0, y_aux0)
    B = clf.predict(X_main)

    clf.fit(X_aux1, y_aux1)
    clf.predict(X_main)
    S = clf.predict(X_main) - B 
    
    combined = main.copy()
    combined['B'] = B 
    combined['S'] = S
        
    return combined

def gradient_boost(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1):

    
    params = {'n_estimators': 500,
          'max_depth': 4,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'}
    
    # Model 
    clf = ensemble.GradientBoostingRegressor(**params)
    
    clf.fit(X_aux0, y_aux0)
    B = clf.predict(X_main)

    clf.fit(X_aux1, y_aux1)
    clf.predict(X_main)
    S = clf.predict(X_main) - B 
    
    combined = main.copy()
    combined['B'] = B 
    combined['S'] = S
        
    return combined

def neural_net(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1):
    
    # Model 
    clf = MLPRegressor(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
    
    clf.fit(X_aux0, y_aux0)
    B = clf.predict(X_main)

    clf.fit(X_aux1, y_aux1)
    clf.predict(X_main)
    S = clf.predict(X_main) - B 
    
    combined = main.copy()
    combined['B'] = B 
    combined['S'] = S
        
    return combined

def ElasticNet(main, X_aux0, y_aux0, X_main, X_aux1, y_aux1):
        
    # Model 
    clf = sklearn.linear_model.ElasticNet()
    
    clf.fit(X_aux0, y_aux0)
    B = clf.predict(X_main)

    clf.fit(X_aux1, y_aux1)
    clf.predict(X_main)
    S = clf.predict(X_main) - B 
    
    combined = main.copy()
    combined['B'] = B 
    combined['S'] = S
        
    return combined